##### Running post processing steps and getting final output.

In [0]:
def final_output(Asset_list,Raw_data_path,RN,NR,features_to_test,model_path,Anomaly_threshold,alpha,beta,low_pass_filter_threshold):
  
  '''This function loads the day level predictions based on anomaly percentage and then applies a low pass filter which takes previous two days               predictions to finally mark a date as anomalous or normal. It has two parameter alpha for previous day and beta for day before previous day
  
  Input :: 1. Pandas dataframe containing the aggregated predictions of Random forest at day level and  prediction based on Anomaly threshold.
            2. alpha -- This is the lag1 which takes the previous day's prediction
            3. beta --  This  is the lag2 which takes the day before previous day's prediction
            4. low_pass_filter_threshold -- This is the final threshold which marks a day level data as anomalous or normal
            
  Output :: Pandas dataframe containing the final result at day level'''
  
  Result_df_final = day_level_result(Asset_list,Raw_data_path,RN,NR,features_to_test,model_path,Anomaly_threshold)
  ## Calling the day_level_result function from the previous file
  
  new_df_lag2 = pd.DataFrame()
  ## Creating empty dataframe to store the final results after applying low pass filter
  
  assets = Result_df_final['Name'].unique()
  ## Extracting the names of the assets
  
  
  for a in assets:
    df_group_lag2 = Result_df_final[Result_df_final.Name==a]
    ### Selecting a particular asset's data
    df_group_lag2 = df_group_lag2.sort_values(by='DATE')
    index_list_lag2 = df_group_lag2.index.values.tolist()
    df_group_lag2.loc[index_list_lag2[0], 'new_y_hat'] = df_group_lag2.loc[index_list_lag2[0], 'y']
    df_group_lag2.loc[index_list_lag2[1], 'new_y_hat'] = df_group_lag2.loc[index_list_lag2[1], 'y']
    
    
    #### Storing first two days predictions
    for i in range(2, len(df_group_lag2)):
      ## Iterating through each prediction for this asset
      
      
      df_group_lag2.loc[index_list_lag2[i], 'new_y_hat'] = df_group_lag2.loc[index_list_lag2[i-1], 'new_y_hat'] + alpha*(df_group_lag2.loc[index_list_lag2[i], 'y'] - df_group_lag2.loc[index_list_lag2[i-1], 'new_y_hat']) + beta*(df_group_lag2.loc[index_list_lag2[i], 'y'] - df_group_lag2.loc[index_list_lag2[i-2], 'new_y_hat'])
      ### This is the low pass fiter formula where alpha and beta are the weights given to the predictions of previous day and the day before previous           day respectively. 
      
    new_df_lag2 = new_df_lag2.append(df_group_lag2)
    ## Saving the low pass result
    
  new_df_lag2['final_prediction'] = new_df_lag2['new_y_hat'].apply(lambda x : -1 if x>=low_pass_filter_threshold   else 1)
  #### creating final predicting predictions where 0 represents that the day has been marked as anomalous and 1 means the day has been marked as normal
  
  Final_result = new_df_lag2[['Name','DATE','final_prediction']]
  ## Selecting the relevant columns
  
  return Final_result

In [0]:
Final_result = final_output(Asset_list,Raw_data_path,RN,NR,features_to_test,model_path,Anomaly_threshold,alpha,beta,low_pass_filter_threshold)
Final_result.head()

In [0]:
Final_result.to_csv(result_path,index = False)